# Raspberry Pi Cameraのデータをサーバに送信する

## 概要

### システム構成

システム構成を次図に示します。`Raspberry Pi`で示された枠内がこのnotebookで構築する対象になります。

![システム構成](img/12-components.png)
<!--
```mermaid
flowchart LR
  subgraph S1[server]
    subgraph K["Apache Kafka"]
      TI(["camera topic"])
    end
  end
  subgraph R["Raspberry Pi"]
    C(["Camera"])
    CC["sensor-picamera\nContainer"]
    C---CC==>TI
  end
```
-->

sensor-picameraコンテナはRaspberry Piに接続されたカメラの画像を取得して、そのデータをKafkaブローカに送信します。

### 前提条件

環境を構築、実行するために必要となる前提条件を示します。

* Raspberry Pi OS (bullseye)
  * Raspberry Pi OS Legacyはサポート対象外になります
* [Raspberry Pi カメラモジュール](https://www.raspberrypi.com/documentation/accessories/camera.html)
  * USBカメラはサポート対象外になります
* docker, docker compose v2
  * カメラ画像を送信するコンテナを実行するために必要となリます

前提条件に関する簡単なチェックを行います。

構築環境のOSをチェックします。次のセルを実行してエラーにならないことを確認して下さい。

In [ ]:
lsb_release -d | grep -e 'Raspbian GNU/Linux 11 (bullseye)'

docker, docker composeが利用可能になっていることを確認します。

In [ ]:
docker version

In [ ]:
docker compose version

## パラメータの指定

### 配置場所 

環境を構築するために必要となる資材を配置するディレクトリを次のセルに指定してください。

In [ ]:
# (例)
# target_dir=$HOME/picamera
# target_dir=/srv/picamera

target_dir=

###　picamera

カメラデータを送信するコンテナsensor-picameraではRaspberry Piカメラでの撮影に[Picamera2](https://github.com/raspberrypi/picamera2)を利用します。ここではPicamera2に設定するパラメータを指定します。

カメラの画像サイズを指定します。画像の幅と高さを(width)x(height)で指定することができます。また典型的なサイズについては文字列で指定することもできます。

* QVGA
  * 320x240
* VGA
  * 640x480
* HD
  * 1280x720
* FHD, 2K
  * 1920x1080
* 4K
  * 3840x2160

In [ ]:
# (例)
# PICAMERA2_SIZE=320x240
# PICAMERA2_SIZE=VGA
# PICAMERA2_SIZE=640x480
# PICAMERA2_SIZE=FHD

PICAMERA2_SIZE=

> `4K`などの大きなサイズの画像を指定する場合は、追加で最大メッセージサイズなどの設定を変更する必要があります。

画像サイズ以外のPicamera2のパラメータに関してもコンテナの環境変数により設定することができます。指定する環境変数名はPicamera2のコンフィギュレーションパラメータを以下のルールで変換したものになります。

* 全て大文字に変換する
* プレフィックスに `PICAMERA2_` をつける
* パラメータ名をプレフィックスの後に続ける
  * 例えばcolour_spaceに対応する環境変数名は`PICAMERA2_COLOUR_SPACE`になる
* stream-specific configurationパラメータはプレフィックスの後にパラメータ名を_でつなげる
  * 例えばsizeに対応する環境変数名は`PICAMERA2_SIZE`になる
* camera controlsのように子要素があるパラメータは、パラメータ名の後に_でつなげる
* またcontrol名のように大文字、小文字が混在している場合は大文字の箇所を_で区切る
  * 例えばcontrol名AwbModeに対応する環境変数名は`PICAMERA2_CONTROLS_AWB_MODE`になる
  
環境変数の設定例を次の表に示します。

|環境変数名|説明|設定例|
|---|---|---|
|PICAMERA2_TRANSFORM_HFLIP|水平反転|PICAMERA2_TRANSFORM_HFLIP=1|
|PICAMERA2_TRANSFORM_VFLIP|垂直反転|PICAMERA2_TRANSFORM_VFLIP=1|
|PICAMERA2_BUFFER_COUNT|バッファセット数|PICAMERA2_BUFFER_COUNT=2|
|PICAMERA2_CONTROLS_FRAME_RATE|フレームレート|PICAMERA2_CONTROLS_FRAME_RATE=5|
|PICAMERA2_CONTROLS_AF_MODE|オートフォーカス|PICAMERA2_CONTROLS_AF_MODE=Continuous|
|PICAMERA2_CONTROLS_AWB_MODE|ホワイトバランス|PICAMERA2_CONTROLS_AWB_MODE=Tungsten|


画像サイズ以外のpicamera2パラメータを設定する場合は次のセルで指定して下さい。

In [ ]:
# (例)
# PICAMERA2_PARAMS="
# PICAMERA2_TRANSFORM_HFLIP=1
# PICAMERA2_TRANSFORM_VFLIP=1
# "

PICAMERA2_PARAMS="
"

### 撮影間隔

カメラ画像を取得する時間間隔（秒）を環境変数`SCHEDULE`に指定して下さい。

In [ ]:
# (例)
# SCHEDULE=60

SCHEDULE=60

`SCHEDULE`を指定しない場合は可能な限り連続的に画像を送信し続けます。

### Kafka

カメラデータの送信先となるKafkaブローカに関するパラメータを指定します。

Kafkaの外部公開ホスト名を指定してください。Raspberry PiからKafkaブローカにアクセスするときは、ここで指定したホスト名（またはIPアドレス）でアクセス出来るように設定する必要があります。

In [ ]:
# (例)
# kafka_host=kafka.example.org
# kafka_host=192.168.10.100

kafka_hostname=

Kafkaブローカのポート番号を指定して下さい。

In [ ]:
# (例)
# kafka_port=9092

kafka_port=9092

カメラデータの送信先となるKafkaのトピック名を指定してください。ここで指定する値は、サーバを構築した際に指定した値と一致したものである必要があります。

In [ ]:
# (例)
# picamera_topic=image-sinetstream-picamera

picamera_topic=image-sinetstream-picamera

## 資材の配置

環境を構築するために必要となる資材を配置します。

資材を配置するディレクトリを作成します。

In [ ]:
mkdir -p $target_dir

コンテナの構成を記述した`docker-compose.yml`を配置します。

In [ ]:
cp docker/docker-compose.yml $target_dir/docker-compose.yml

配置した`docker-compose.yml`の内容を表示します。

In [ ]:
cat $target_dir/docker-compose.yml

コンテナの環境変数を設定する`.env`ファイルを作成します。

In [ ]:
cat > $target_dir/.env <<EOF
PICAMERA2_SIZE=${PICAMERA2_SIZE:?Image size must be set.}
${PICAMERA2_PARAMS:-}
SS_BROKERS=${kafka_hostname:?The hostname of Kafka must be set.}:${kafka_port:-9092}
SS_TOPIC=${picamera_topic:?The topic name must be set.}
SS_CONSISTENCY=AT_LEAST_ONCE
EOF

In [ ]:
if [ -n "$SCHEDULE" ]; then
cat >> $target_dir/.env <<EOF
SCHEDULE=$SCHEDULE
EOF
fi

`.env`ファイルの内容を確認します。

In [ ]:
cat $target_dir/.env

## コンテナの起動

利用するコンテナイメージを取得します。

In [ ]:
docker compose --project-directory ${target_dir} pull -q

コンテナを起動します。notebook環境で`docker compose up`を実行すると処理中の表示が煩雑なため、次のセルでは全ての出力結果を破棄しています。エラーや警告表示を確認する必要がある場合はnotebook環境ではなく、別窓でターミナルなどを開いて`docker compose up`コマンドを実行して下さい。

In [ ]:
docker compose --project-directory ${target_dir} up -d --remove-orphans >& /dev/null

コンテナの実行状況を確認します。コンテナのSTATUSがUpとなっていることを確認して下さい。

In [ ]:
docker compose --project-directory ${target_dir} ps